In [1]:
# add path to the directory containing the module
import sys
sys.path.append('../../src')

from transformers import RobertaTokenizerFast
roberta_path = "../../models/roberta-base-finetune/roberta-base-finetune-2024-05-20_08-02-29-65707/checkpoint-16482"
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

In [1]:
num_layers = 1

In [2]:
for _ in range(num_layers):
    print("X")

X


In [2]:
from preprocessing.pre_processor import PreProcessor

In [41]:
preprocessor = PreProcessor(
    tokenizer,
    batch_size=16,
    max_sentences_per_article=32,
    max_sentence_length=64,
    max_args_per_sentence=3,
    max_arg_length=16,
    test_size=0.1,
    frameaxis_dim=10,
    bert_model_name="roberta-base",
    name_tokenizer="roberta-base",
    path_name_bert_model=roberta_path,
    path_antonym_pairs="../../data/axis/mft.json",
    dim_names=["virtue", "vice"],
)

In [49]:
# read json file using pandas
import pandas as pd

df = pd.read_json("../../data/mfc/immigration_labeled_preprocessed.json")

df.columns

Index(['article_id', 'text', 'document_frame', 'Capacity and Resources',
       'Crime and Punishment', 'Cultural Identity', 'Economic',
       'External Regulation and Reputation', 'Fairness and Equality',
       'Health and Safety', 'Legality, Constitutionality, Jurisdiction',
       'Morality', 'Other', 'Policy Prescription and Evaluation', 'Political',
       'Public Sentiment', 'Quality of Life', 'Security and Defense'],
      dtype='object')

In [42]:
# add path to the directory containing the module
import sys
sys.path.append('../../src')

from transformers import RobertaTokenizerFast, RobertaModel
roberta_path = "../../models/roberta-base-finetune/roberta-base-finetune-2024-05-20_08-02-29-65707/checkpoint-16482"
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained(roberta_path)

Some weights of the model checkpoint at ../../models/roberta-base-finetune/roberta-base-finetune-2024-05-20_08-02-29-65707/checkpoint-16482 were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../../models/roberta-base-finetune/roberta-base-finetune-2024-05-20_08-02-29-65707/checkpoint-16482 and are

In [2]:
from model.slmuse_dlf.srl_embeddings import SRLEmbeddings

In [3]:
srl = SRLEmbeddings(roberta_path, "roberta-base", pooling="mean")

Some weights of the model checkpoint at ../../models/roberta-base-finetune/roberta-base-finetune-2024-05-20_08-02-29-65707/checkpoint-16482 were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../../models/roberta-base-finetune/roberta-base-finetune-2024-05-20_08-02-29-65707/checkpoint-16482 and are

In [97]:
example_setence = [
    "The quick brown fox jumps over the lazy dog.",
    "Hello my name is elias",
    "I am 26 years old",
    "The orange cat is sleeping on the couch.",
    "I like to play soccer",
    "I am from Germany",
]

In [98]:
output = tokenizer(example_setence, add_special_tokens=True,
                max_length=16,
                truncation=True,
                padding="max_length",
                return_attention_mask=True,
                return_tensors="pt")

In [102]:
sentence_ids = output["input_ids"].unsqueeze(0)
attention_masks = output["attention_mask"].unsqueeze(0)

In [109]:
sentence_ids = sentence_ids.reshape(2, 3, -1)
attention_masks = attention_masks.reshape(2, 3, -1)

In [110]:
own_embedding = model(**output)

In [141]:
sentence_ids[0]

tensor([[    0,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335,
             4,     2,     1,     1,     1,     1],
        [    0, 31414,   127,   766,    16,  1615,  5003,     2,     1,     1,
             1,     1,     1,     1,     1,     1],
        [    0,   100,   524,   973,   107,   793,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1]])

In [112]:
tokenizer.decode([22414,  2335])

' lazy dog'

In [185]:
import torch

predicate_ids = torch.tensor(
    [[[13855, 0, 0, 0, 0], [2119, 6219, 0, 0, 0]], [[1615, 0, 0, 0, 0], [5003, 2, 0, 0, 0]], [[793, 0, 0, 0, 0], [973, 107, 0, 0, 0]],
     [[8978, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[101, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[1600, 0, 0, 0, 0], [0, 0, 0, 0, 0]]]
).reshape(2, 3, -1, 5)

arg0_ids = torch.tensor(
    [[[133, 2119, 6219, 23602, 0], [81,5, 0, 0, 0]], [[127, 0, 0, 0, 0], [127,766, 0, 0, 0]], [[100, 0, 0, 0, 0], [100,524, 0, 0, 0]],
     [[16, 8416, 0, 0, 0], [0, 0, 0, 0, 0]], [[7, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[31, 1600, 0, 0, 0], [0, 0, 0, 0, 0]]]
).reshape(2, 3, -1, 5)

arg1_ids = torch.tensor(
    [[[22414, 2335, 0, 0, 0], [81,5, 0, 0, 0]], [[16, 0, 0, 0, 0], [16,1615, 0, 0, 0]], [[973, 0, 0, 0, 0], [793,2, 0, 0, 0]],
     [[15, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[100, 101, 0, 0, 0], [0, 0, 0, 0, 0]], [[524, 0, 0, 0, 0], [0, 0, 0, 0, 0]]]
).reshape(2, 3, -1, 5)


In [186]:
sentence_ids.shape, attention_masks.shape, predicate_ids.shape, arg0_ids.shape, arg1_ids.shape

(torch.Size([2, 3, 16]),
 torch.Size([2, 3, 16]),
 torch.Size([2, 3, 2, 5]),
 torch.Size([2, 3, 2, 5]),
 torch.Size([2, 3, 2, 5]))

In [187]:
srl_output = srl(sentence_ids, attention_masks, predicate_ids, arg0_ids, arg1_ids)

c:\Users\elias\anaconda3\envs\nlp310\lib\site-packages\torch\amp\autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Found for 13855 at tensor([[5]]) the following embedding tensor([[ 0.0269,  0.2135,  0.0282, -0.1157, -0.0593]])
Found 1 embeddings for arg 0
Embedding after cat: torch.Size([1, 768]), tensor([ 0.0269,  0.2135,  0.0282, -0.1157, -0.0593])
Avg embedding: torch.Size([768]), tensor([ 0.0269,  0.2135,  0.0282, -0.1157, -0.0593])
Found for 2119 at tensor([[2]]) the following embedding tensor([[-0.0161,  0.1466, -0.0592, -0.0198,  0.2405]])
Found for 6219 at tensor([[3]]) the following embedding tensor([[-0.0501, -0.0774,  0.0512,  0.1610,  0.6101]])
Found 2 embeddings for arg 1
Embedding after cat: torch.Size([2, 768]), tensor([-0.0161,  0.1466, -0.0592, -0.0198,  0.2405])
Avg embedding: torch.Size([768]), tensor([-0.0331,  0.0346, -0.0040,  0.0706,  0.4253])
Found for 1615 at tensor([[5]]) the following embedding tensor([[-0.1381, -0.1109,  0.0400, -0.1476, -0.7762]])
Found 1 embeddings for arg 0
Embedding after cat: torch.Size([1, 768]), tensor([-0.1381, -0.1109,  0.0400, -0.1476, -0.7762

In [188]:
(
    sentence_embeddings_avg,
    predicate_embeddings,
    arg0_embeddings,
    arg1_embeddings,
) = srl_output

In [189]:
predicate_embeddings.shape

torch.Size([2, 3, 2, 768])

In [195]:
predicate_embeddings[:, :, 0]

tensor([[[ 0.0269,  0.2135,  0.0282,  ...,  0.1282, -0.0673,  0.1796],
         [-0.1381, -0.1109,  0.0400,  ...,  0.2553, -0.0297,  0.3096],
         [ 0.1847, -0.2344,  0.2896,  ..., -0.2153, -0.0164,  0.0806]],

        [[-0.0154, -0.0298,  0.1042,  ..., -0.0728, -0.1524,  0.2402],
         [-0.0568,  0.2555,  0.2279,  ...,  0.7190,  0.1341, -0.3069],
         [-0.1040, -0.0363,  0.1299,  ...,  0.3633,  0.0034, -0.0800]]])

In [191]:
def negative_sampling(embeddings, num_negatives=-1):
    if num_negatives == -1:
        num_negatives = embeddings.size(0)

    batch_size, num_sentences, num_args, embedding_dim = embeddings.size()
    all_negatives = []

    for i in range(batch_size):
        for j in range(num_sentences):
            # Flatten the arguments dimension to sample across all arguments in the sentence
            flattened_embeddings = embeddings[i, j].view(-1, embedding_dim)

            # Get indices of non-padded embeddings (assuming padding is represented by all-zero vectors)
            non_padded_indices = torch.where(
                torch.any(flattened_embeddings != 0, dim=1)
            )[0]

            # Randomly sample negative indices from non-padded embeddings
            if len(non_padded_indices) > 0:
                num_samples = min(num_negatives, len(non_padded_indices))
                negative_indices = non_padded_indices[
                    torch.randperm(len(non_padded_indices))[:num_samples]
                ]
                negative_samples = flattened_embeddings[negative_indices, :]
                all_negatives.append(negative_samples)

    if len(all_negatives) == 0:
        return torch.zeros((num_negatives, embedding_dim), device=embeddings.device)

    # Concatenate all negative samples into a single tensor
    all_negatives = torch.cat(all_negatives, dim=0)

    # Ensure we have the right number of negatives
    if all_negatives.size(0) > num_negatives:
        indices = torch.randperm(all_negatives.size(0))[:num_negatives]
        all_negatives = all_negatives[indices]

    return all_negatives

In [192]:
negative_sampling(predicate_embeddings, num_negatives=-1)

tensor([[ 0.0138,  0.2253, -0.0590,  ..., -0.2056, -0.0329,  0.0414],
        [-0.0331,  0.0346, -0.0040,  ...,  0.0861, -0.0661,  0.1827]])

In [193]:
predicate_embeddings.shape

torch.Size([2, 3, 2, 768])

In [1]:
import torch

# Example where logits_p has identical values
logits_p = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]])


# Apply softmax
d_p = torch.softmax(logits_p, dim=1)

# Check mean and std
mean_d_p = d_p.mean().item()
std_d_p = d_p.std().item()
sum_d_p = d_p.sum().item()

print(f"mean: {mean_d_p}, std: {std_d_p}, sum: {sum_d_p}")


mean: 0.06666667014360428, std: 0.0, sum: 1.0


In [21]:
# create random tensor of shape [2, 3, 768]
x = torch.randn(2, 3, 768)
# create random tensor of shape [2, 3, 768]

In [22]:
# set [0, 1,] to 0
x[0, 1] = 0

In [23]:
mask = (x != 0).any(dim=-1)

In [24]:
mask.unsqueeze(-1).float()

tensor([[[1.],
         [0.],
         [1.]],

        [[1.],
         [1.],
         [1.]]])

In [25]:
x = x * mask.unsqueeze(-1).float()

In [2]:
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
import io
import torch

In [3]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

In [10]:
# load frameaxis data
with open(f'../../data/frameaxis/mfc/frameaxis_mft.pkl', 'rb') as f:
    df_frameaxis = CPU_Unpickler(f).load()

df_mfc = pd.DataFrame(json.load(open("../../data/mfc/immigration_labeled_preprocessed.json")))

In [14]:
df_frameaxis.groupby("article_id").size().sort_values(ascending=False).equals(df_mfc.groupby("article_id").size().sort_values(ascending=False))


True

In [6]:
df_frameaxis[df_frameaxis["article_id"] == "Immigration1.0-10005"]

,article_id,authority_bias,authority_intensity,care_bias,care_intensity,fairness_bias,fairness_intensity,loyalty_bias,loyalty_intensity,sanctity_bias,sanctity_intensity
0,Immigration1.0-10005,0.119627,0.001947,0.121117,0.002917,0.049433,0.002096,-0.083773,0.001122,0.039550,0.000685
1,Immigration1.0-10005,0.042660,0.002332,0.047766,0.002328,-0.003897,0.000673,-0.113696,0.002255,0.007318,0.001060
2,Immigration1.0-10005,0.080072,0.002810,0.086251,0.001226,0.023373,0.000814,-0.092979,0.001367,0.018394,0.000473
3,Immigration1.0-10005,0.030022,0.002428,0.043807,0.002104,0.001129,0.000178,-0.154950,0.003740,0.021082,0.001083
4,Immigration1.0-10005,0.025590,0.002913,0.022914,0.003412,-0.001933,0.000334,-0.168797,0.004817,-0.027492,0.002658
5,Immigration1.0-10005,0.123118,0.007518,0.124388,0.005661,0.030050,0.002975,-0.060810,0.003297,0.042067,0.001848
6,Immigration1.0-10005,0.092428,0.001526,0.103190,0.001681,0.017554,0.000366,-0.088889,0.002141,0.031814,0.000551
7,Immigration1.0-10005,0.075311,0.000351,0.096394,0.000607,0.013698,0.000856,-0.061781,0.002005,0.030850,0.000602
8,Immigration1.0-10005,0.088887,0.000839,0.106613,0.002163,-0.007102,0.002222,-0.093328,0.000599,0.039617,0.000882
9,Immigration1.0-10005,0.071010,0.001132,0.078965,0.000822,-0.026710,0.003072,-0.116941,0.001069,-0.009133,0.001732


In [9]:
df_mfc[df_mfc["article_id"] == "Immigration1.0-10005"]

,article_id,text,document_frame,Capacity and Resources,Crime and Punishment,Cultural Identity,Economic,External Regulation and Reputation,Fairness and Equality,Health and Safety,"Legality, Constitutionality, Jurisdiction",Morality,Other,Policy Prescription and Evaluation,Political,Public Sentiment,Quality of Life,Security and Defense
0,Immigration1.0-10005,Immigrants without HOPE need help entering col...,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Immigration1.0-10005,Anxiety gripped Jose Alvarado in third-period ...,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Immigration1.0-10005,It mounted as students went around the room te...,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Immigration1.0-10005,Georgia Tech.,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Immigration1.0-10005,University of Georgia.,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,Immigration1.0-10005,"""All I could say was, 'I'm planning to see if ...",Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
6,Immigration1.0-10005,Alvarado earned a 3.5 grade-point average and ...,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7,Immigration1.0-10005,He led the soccer team and was in the National...,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,Immigration1.0-10005,"Until recently, though, college seemed out of ...",Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,Immigration1.0-10005,That's because when Jose Alvarado was four mon...,Quality of Life,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [53]:
v_span = torch.randn(8, 768)
v_span[2] = 0

In [54]:
mask_span = (v_span.abs().sum(dim=-1) != 0).float().bool()

In [55]:
mask_span

tensor([ True,  True, False,  True,  True,  True,  True,  True])

In [56]:
mask_span.unsqueeze(-1).float()

tensor([[1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [3]:
from transformers import RobertaTokenizerFast, RobertaModel

In [4]:
sentecence = "The quick brown fox jumps over the lazy dog."
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
output = model(**tokenizer(sentecence, return_tensors="pt"), output_hidden_states=True)

In [7]:
output.hidden_states

(tensor([[[ 0.1664, -0.0541, -0.0014,  ..., -0.0811,  0.0794,  0.0155],
          [-0.1961,  0.0757,  0.0936,  ..., -0.1657, -0.1581,  0.2045],
          [-0.2031,  0.6899,  0.0311,  ..., -0.2548,  0.0058, -0.2097],
          ...,
          [-0.2866,  0.1725,  0.3406,  ..., -0.3028,  0.4116,  0.2430],
          [ 0.3401,  0.1489,  0.1269,  ..., -0.8637,  0.0431,  0.1276],
          [ 0.0731,  0.0143,  0.0267,  ...,  0.4326,  0.0403, -0.0697]]],
        grad_fn=<NativeLayerNormBackward0>),
 tensor([[[-0.0142,  0.0260,  0.0163,  ..., -0.0147,  0.0251, -0.1151],
          [-0.2826, -0.1724,  0.2660,  ..., -0.4618, -0.0659,  0.5169],
          [-0.0227,  1.1936,  0.0787,  ..., -0.6904, -0.0903, -0.1355],
          ...,
          [-0.5332,  0.6504,  0.0086,  ..., -0.3928,  0.7444,  0.3632],
          [ 0.2366,  0.2497,  0.3293,  ..., -0.4577,  0.1939,  0.1081],
          [-0.2670,  0.2685,  0.1422,  ...,  0.5637,  0.0469, -0.5314]]],
        grad_fn=<NativeLayerNormBackward0>),
 tensor([[[ 

In [1]:
import os

In [10]:
! export TEST="Capacity and Resources;Crime and Punishment;Cultural Identity;Economic;External Regulation and Reputation;Fairness and Equality;Health and Safety;Legality, Constitutionality, Jurisdiction;Morality;Other;Policy Prescription and Evaluation;Political;Public Sentiment;Quality of Life;Security and Defense"

Der Befehl "export" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [12]:
import os

# Define the list of strings
categories = ['Capacity_and_resources', 'Crime_and_punishment', 'Cultural_identity', 'Economic',
                 'External_regulation_and_reputation', 'Fairness_and_equality', 'Health_and_safety',
                 'Legality_Constitutionality_and_jurisprudence', 'Morality', 'Policy_prescription_and_evaluation',
                 'Political', 'Public_opinion', 'Quality_of_life', 'Security_and_defense']

# Join the list into a single string with a delimiter
delimiter = ';'  # Using semicolon as a delimiter
encoded_categories = delimiter.join(categories)

# Set the environment variable
os.environ['MY_CATEGORIES'] = encoded_categories

# Verify it's set
print("Environment variable set as: ", os.getenv('MY_CATEGORIES'))

Environment variable set as:  Capacity_and_resources;Crime_and_punishment;Cultural_identity;Economic;External_regulation_and_reputation;Fairness_and_equality;Health_and_safety;Legality_Constitutionality_and_jurisprudence;Morality;Policy_prescription_and_evaluation;Political;Public_opinion;Quality_of_life;Security_and_defense


: 

In [11]:
# Get the environment variable
encoded_categories = os.getenv('MY_CATEGORIES')

if encoded_categories:
    # Split the string back into a list
    categories = encoded_categories.split(delimiter)
    print("Decoded list: ", categories)
else:
    print("Environment variable not set.")


Decoded list:  ['Capacity and Resources', 'Crime and Punishment', 'Cultural Identity', 'Economic', 'External Regulation and Reputation', 'Fairness and Equality', 'Health and Safety', 'Legality, Constitutionality, Jurisdiction', 'Morality', 'Other', 'Policy Prescription and Evaluation', 'Political', 'Public Sentiment', 'Quality of Life', 'Security and Defense']


In [2]:
import torch
from transformers import RobertaModel, RobertaTokenizer

# Load pre-trained RoBERTa model and tokenizer
model_name = 'roberta-base'
model = RobertaModel.from_pretrained(model_name)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# Example sentences
sentence1 = "The quick brown fox jumps over the lazy dog."
sentence2 = "A fast, dark-colored fox leaps over a sleeping dog."

# Tokenize and get embeddings
inputs1 = tokenizer(sentence1, return_tensors='pt')
inputs2 = tokenizer(sentence2, return_tensors='pt')

with torch.no_grad():
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

# Get the embeddings (using the last hidden state for simplicity)
embedding1 = outputs1.last_hidden_state.mean(dim=1)
embedding2 = outputs2.last_hidden_state.mean(dim=1)

# Calculate L2 norm using torch.linalg.vector_norm
l2_norm = torch.linalg.vector_norm(embedding1 - embedding2).item()
print(f"L2 norm between embeddings: {l2_norm}")


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


L2 norm between embeddings: 1.90340256690979


In [4]:
def l2(u, v):
    # Calculate L2 distance and ensure no negative values are passed to sqrt
    dist_squared = torch.sum((u - v) ** 2, dim=1)
    dist_squared = torch.clamp(dist_squared, min=0)  # Ensure non-negative values
    return torch.sqrt(dist_squared + 1e-8)  # Add epsilon to avoid sqrt(0)

In [5]:
l2_norm_test = l2(embedding1, embedding2).item()

In [37]:
# Example one-hot encoded labels and predictions
one_hot_labels = torch.tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                               [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                               [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                               [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.long)

# Example raw logits for a batch of 8 samples and 15 classes
logits = torch.randn(8, 15)  # Random logits for demonstration

In [38]:
import torch
import torch.nn as nn

# Convert one-hot encoded labels to class indices
labels = torch.argmax(one_hot_labels, dim=1).long()

# Initialize CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

# Calculate the loss
loss = criterion(logits, labels)
loss_2 = criterion(logits, one_hot_labels.softmax(dim=1))
print(f'Loss: {loss.item()}')
print(f'Loss 2: {loss_2.item()}')


RuntimeError: "softmax_lastdim_kernel_impl" not implemented for 'Long'

In [21]:
import torch
import torch.nn as nn

# Example raw logits for a batch of 3 samples and 5 classes
logits = torch.randn(3, 5, requires_grad=True)

# Example class probabilities (soft targets) for each sample
target = torch.randn(3, 5).softmax(dim=1)

# Initialize CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

# Calculate the loss
loss = criterion(logits, target)
loss.backward()

print(f'Loss with class probabilities: {loss.item()}')


Loss with class probabilities: 2.3973052501678467


In [1]:
import torch

In [2]:
# create tensor of shape [batch_size, num_classes]
logits = torch.randn(3, 5)

In [8]:
torch.softmax(logits, dim=1)[0].sum()

tensor(1.)